In [13]:
from v_diffusion import *
import torch
from torchvision.utils import save_image

model = UNet(
    in_channels=3,
    hid_channels=128,
    out_channels=6,
    ch_multipliers=[1, 2, 3, 4],
    num_res_blocks=5,
    apply_attn=[False, True, True, True],
    drop_rate=0.1,
    #num_classes=10
)

path = "./chkpts/FID/" + "dist_64_128_lsun_bedroom_128_dist_1_5000.pt"

device = "cuda:0"
chkpt = torch.load(path, map_location=device)["model"]#["ema"]["shadow"]
print(chkpt.keys())
model.to(device)
# for k in list(chkpt.keys()):
#     if k.startswith("module."):
#         chkpt[k.split(".", maxsplit=1)[1]] = chkpt.pop(k)
model.load_state_dict(chkpt)
model.eval()
model.requires_grad_(False)
del chkpt

logsnr_fn = get_logsnr_schedule("cosine", -20, 20)
diffusion = GaussianDiffusion(
    logsnr_fn,
    sample_timesteps=128,
    model_out_type="both",
    model_var_type="fixed_large",
    reweight_type="truncated_snr",
    loss_type="mse",
    w_guide=0.1,
    p_uncond=0.1)


odict_keys(['time_embed.0.weight', 'time_embed.0.bias', 'time_embed.2.weight', 'time_embed.2.bias', 'class_embed.1.weight', 'class_embed.1.bias', 'in_conv.weight', 'in_conv.bias', 'downsamples.level_0.0.norm1.weight', 'downsamples.level_0.0.norm1.bias', 'downsamples.level_0.0.conv1.weight', 'downsamples.level_0.0.conv1.bias', 'downsamples.level_0.0.fc.weight', 'downsamples.level_0.0.fc.bias', 'downsamples.level_0.0.norm2.weight', 'downsamples.level_0.0.norm2.bias', 'downsamples.level_0.0.conv2.weight', 'downsamples.level_0.0.conv2.bias', 'downsamples.level_0.1.norm1.weight', 'downsamples.level_0.1.norm1.bias', 'downsamples.level_0.1.conv1.weight', 'downsamples.level_0.1.conv1.bias', 'downsamples.level_0.1.fc.weight', 'downsamples.level_0.1.fc.bias', 'downsamples.level_0.1.norm2.weight', 'downsamples.level_0.1.norm2.bias', 'downsamples.level_0.1.conv2.weight', 'downsamples.level_0.1.conv2.bias', 'downsamples.level_0.2.norm1.weight', 'downsamples.level_0.2.norm1.bias', 'downsamples.level

RuntimeError: Error(s) in loading state_dict for UNet:
	Missing key(s) in state_dict: "downsamples.level_0.3.norm1.weight", "downsamples.level_0.3.norm1.bias", "downsamples.level_0.3.conv1.weight", "downsamples.level_0.3.conv1.bias", "downsamples.level_0.3.fc.weight", "downsamples.level_0.3.fc.bias", "downsamples.level_0.3.norm2.weight", "downsamples.level_0.3.norm2.bias", "downsamples.level_0.3.conv2.weight", "downsamples.level_0.3.conv2.bias", "downsamples.level_0.4.norm1.weight", "downsamples.level_0.4.norm1.bias", "downsamples.level_0.4.conv1.weight", "downsamples.level_0.4.conv1.bias", "downsamples.level_0.4.fc.weight", "downsamples.level_0.4.fc.bias", "downsamples.level_0.4.norm2.weight", "downsamples.level_0.4.norm2.bias", "downsamples.level_0.4.conv2.weight", "downsamples.level_0.4.conv2.bias", "downsamples.level_0.5.norm1.weight", "downsamples.level_0.5.norm1.bias", "downsamples.level_0.5.conv1.weight", "downsamples.level_0.5.conv1.bias", "downsamples.level_0.5.fc.weight", "do

In [26]:

ws = [3]
# torch.manual_seed(1234)
noise = torch.randn((1, 3, 64, 64)).to(device)

for w in ws:
    diffusion.w_guide = w

    x_gen = diffusion.p_sample(
        model, (1, 3, 64, 64), device=device,
        noise=noise,
        # label=torch.arange(
        #     1, 11, device=device, dtype=torch.float32).repeat_interleave(10),
        # label=torch.tensor(
        #     0, device=device, dtype=torch.float32),
        label=None,
        use_ddim=True)

    save_image(
        x_gen.cpu(),
        f"./cifar10_w{w}.png",
        nrow=1,
        normalize=True,
        value_range=(-1, 1)
    )